In [20]:
from collections import defaultdict
import glob
import json
import pathlib

# Define the pattern to match the files
pattern = r'D:\BEHAVIOR-1K\asset_pipeline\cad\*\*\artifacts\replaced_bad_objects.json'

# Initialize an empty dictionary to hold the merged contents
merged_list = []

# Find all files matching the pattern
file_paths = glob.glob(pattern)

# Iterate over each file and merge its contents into the dictionary
for file_path in file_paths:
  target = "/".join(pathlib.Path(file_path).parts[-4:-2])
  with open(file_path, 'r') as file:
    data = json.load(file)
    for model, instances in data.items():
      for instance_id, instance in instances.items():
        merged_list.append(((target, model, instance_id), instance))

metadata, infos = zip(*merged_list)

In [44]:
infos[0].keys()

dict_keys(['original_transform', 'calculated_transform', 'current_transform', 'original_world_bb', 'computed_world_bb', 'current_world_bb', 'original_lowbb', 'current_lowbb'])

In [26]:
# Load the inventory so that we can find out the category of each model ID
with open(r'D:\BEHAVIOR-1K\asset_pipeline\artifacts\pipeline\object_inventory.json', 'r') as file:
  inventory = json.load(file)

category_by_model_id = {entry.split("-")[1]: entry.split("-")[0] for entry in inventory["providers"]}

In [61]:
import numpy as np

def extract_size_differences(extract_current_value, extract_baseline_value, subtract_values, reporting_threshold):
  current_values = np.array([extract_current_value(info) for info in infos])
  baseline_values = np.array([extract_baseline_value(info) for info in infos])
  value_diffs = subtract_values(current_values, baseline_values)

  # Flatten the world bb size differences and describe the statistics
  print("Stats:")
  print("  Shape", value_diffs.shape)
  print("  Mean", np.mean(value_diffs))
  print("  Std", np.std(value_diffs))
  print("  Min", np.min(value_diffs))
  print("  Max", np.max(value_diffs))

  # Print the value of each percentile
  for i in range(0, 101, 10):
    print(f'  {i}th percentile: {np.percentile(value_diffs, i)}')

  # How many items contain any deltas that are greater than 10mm apart?
  bad_items = value_diffs > reporting_threshold
  bad_item_indices = np.where(bad_items)[0]
  bad_item_metas = [metadata[i] for i in bad_item_indices]
  bad_item_infos = [infos[i] for i in bad_item_indices]
  bad_items = list(zip(bad_item_metas, bad_item_infos))
  print("\nTotal count of bad items:", len(bad_items))

  # Group those by target
  bad_items_by_target = defaultdict(list)
  for meta, info in bad_items:
    target = meta[0]
    dist = subtract_values(extract_current_value(info), extract_baseline_value(info))
    bad_items_by_target[target].append((meta, dist))

  # Print each bad item per target sorted by how bad it is
  for target, bad_items in bad_items_by_target.items():
    print()
    print(target)
    bad_items.sort(key=lambda x: x[1], reverse=True)
    for meta, dist in bad_items:
      model_id = meta[1]
      instance = meta[2]
      category = category_by_model_id.get(model_id, "Unknown")
      print(f'{category}-{model_id}-{instance}: {dist:.4f}')

In [62]:
# Run for sizes
current_size_extractor = lambda info: np.array(info['current_world_bb'][1]) - np.array(info['current_world_bb'][0])
baseline_size_extractor = lambda info: np.array(info['original_world_bb'][1]) - np.array(info['original_world_bb'][0])
subtract_values = lambda current, baseline: np.max(np.abs(current - baseline), axis=-1)
extract_size_differences(current_size_extractor, baseline_size_extractor, subtract_values, 10)

Stats:
  Shape (1486,)
  Mean 0.7387161116300008
  Std 3.980667794750501
  Min 0.0
  Max 96.3349609375
  0th percentile: 0.0
  10th percentile: 2.276897430419922e-05
  20th percentile: 6.103515625e-05
  30th percentile: 0.0001220703125
  40th percentile: 0.000244140625
  50th percentile: 0.00048828125
  60th percentile: 0.0009765625
  70th percentile: 0.0106201171875
  80th percentile: 0.21435546875
  90th percentile: 1.4149169921875
  100th percentile: 96.3349609375

Total count of bad items: 25

scenes/Beechwood_0_garden
top_cabinet-fqhdne-0: 27.9336
fridge-dszchb-0: 14.1367

scenes/Beechwood_0_int
countertop-tpuwys-1: 96.3350
top_cabinet-fqhdne-0: 27.9326
fridge-dszchb-0: 14.1362

scenes/Beechwood_1_int
top_cabinet-fqhdne-0: 11.7798

scenes/Benevolence_1_int
top_cabinet-lsyzkh-0: 10.1700

scenes/Pomaria_0_int
shelf-vgzdul-0: 14.8743
shelf-vgzdul-1: 12.5293

scenes/Pomaria_1_int
top_cabinet-fqhdne-0: 25.0264

scenes/Rs_garden
microwave-abzvij-0: 11.4700

scenes/Rs_int
microwave-abzvi

In [63]:
# Now do the same for centers
current_center_extractor = lambda info: np.array(info['current_world_bb'][1]) + np.array(info['current_world_bb'][0]) / 2.
baseline_center_extractor = lambda info: np.array(info['original_world_bb'][1]) + np.array(info['original_world_bb'][0]) / 2.
subtract_values = lambda current, baseline: np.max(np.abs(current - baseline), axis=-1)
extract_size_differences(current_center_extractor, baseline_center_extractor, subtract_values, 10)

Stats:
  Shape (1486,)
  Mean 0.18648382878961345
  Std 0.9967486106489379
  Min 0.0
  Max 24.083740234375
  0th percentile: 0.0
  10th percentile: 1.1920928955078125e-05
  20th percentile: 4.57763671875e-05
  30th percentile: 0.0001220703125
  40th percentile: 0.0001361370086669922
  50th percentile: 0.000244140625
  60th percentile: 0.00048828125
  70th percentile: 0.001953125
  80th percentile: 0.05804443359375
  90th percentile: 0.360626220703125
  100th percentile: 24.083740234375

Total count of bad items: 2

scenes/Beechwood_0_int
countertop-tpuwys-1: 24.0837

scenes/Wainscott_1_int
top_cabinet-fqhdne-0: 12.3848


In [68]:
# Finally lets do it for the rotation matrix
from scipy.spatial.transform import Rotation as R
_EPS = np.finfo(float).eps * 4.0
def get_rotation_from_transform(matrix):
    M = np.array(matrix, dtype=np.float64).T
    if abs(M[3, 3]) < _EPS:
        raise ValueError("M[3, 3] is zero")
    M /= M[3, 3]
    P = M.copy()
    P[:, 3] = 0.0, 0.0, 0.0, 1.0
    if not np.linalg.det(P):
        raise ValueError("matrix is singular")

    scale = np.zeros((3,))
    shear = [0.0, 0.0, 0.0]

    if any(abs(M[:3, 3]) > _EPS):
        M[:, 3] = 0.0, 0.0, 0.0, 1.0

    M[3, :3] = 0.0

    row = M[:3, :3].copy()
    scale[0] = np.linalg.norm(row[0])
    row[0] /= scale[0]
    shear[0] = np.dot(row[0], row[1])
    row[1] -= row[0] * shear[0]
    scale[1] = np.linalg.norm(row[1])
    row[1] /= scale[1]
    shear[0] /= scale[1]
    shear[1] = np.dot(row[0], row[2])
    row[2] -= row[0] * shear[1]
    shear[2] = np.dot(row[1], row[2])
    row[2] -= row[1] * shear[2]
    scale[2] = np.linalg.norm(row[2])
    row[2] /= scale[2]
    shear[1:] /= scale[2]

    if np.dot(row[0], np.cross(row[1], row[2])) < 0:
        np.negative(scale, scale)
        np.negative(row, row)

    return row.T, scale

current_rotation_extractor = lambda info: get_rotation_from_transform(info['current_transform'])[0]
baseline_rotation_extractor = lambda info: get_rotation_from_transform(info['original_transform'])[0]
subtract_values = lambda current, baseline: np.array([np.rad2deg((R.from_matrix(b).inv() * R.from_matrix(c)).magnitude()) for b, c in zip(baseline, current)])
extract_size_differences(current_rotation_extractor, baseline_rotation_extractor, subtract_values, 1)

Stats:
  Shape (1486,)
  Mean 0.0027136679440227995
  Std 0.016339414803300166
  Min 0.0
  Max 0.3195149144442973
  0th percentile: 0.0
  10th percentile: 6.376479618472306e-14
  20th percentile: 4.3820510726244904e-13
  30th percentile: 3.413470242769857e-11
  40th percentile: 4.040315682753605e-09
  50th percentile: 3.452541863470996e-08
  60th percentile: 3.3337107951295256e-06
  70th percentile: 5.122643989398504e-06
  80th percentile: 1.104036237651266e-05
  90th percentile: 0.0015844999831939092
  100th percentile: 0.3195149144442973

Total count of bad items: 0


In [43]:
# Search for instances of vgzdul and get their world BB size differences
bad_shelf = "vgzdul"
bad_fridge = "dszchb"
[(meta, np.max(np.abs(np.array(info['current_world_bb']) - np.array(info["original_world_bb"]))))
 for meta, info in merged_list if meta[1] == bad_fridge]

[(('scenes/Beechwood_0_garden', 'dszchb', '0'), 7.068359375),
 (('scenes/Beechwood_0_int', 'dszchb', '0'), 7.068115234375),
 (('scenes/Wainscott_0_garden', 'dszchb', '0'), 7.75439453125),
 (('scenes/Wainscott_0_int', 'dszchb', '0'), 7.75390625)]